# Inverse Optimal Tax Example

This notebook provides an illustrative example for workint with `iot` an inverse optimal tax model.  This model infers a generalized social welfare function from a specification of tax policy.  The underlying assumption is that those choosing the tax policy are acting optimally, i.e., they are trying to maximize their social welfare function.  This assumption of optimizing a social welfare function, together with data and empirical estimates of behavioral parameters, can be used to reverse engineer what the underyling social welfare function would have to look like if indeed tax policy were trying to maximize it.

## Step 1: Import the `iot_comparison` class object

The `iot_comparison` class object is the high-level user interfaces for the `iot` model.  It will allow the user to specify tax policies to evaluate and provide a number of parameters for the `iot` model, such as the underlying data to use, how marginal tax rates are to be calculated, and more.

Note thaat the `iot_comparison` class implicitly assumes one is working with the [Tax-Calculator](https://github.com/PSLmodels/Tax-Calculator/) model.  Therefore, the specification of tax policy should follow the format for a parametric reform in Tax-Calculator.

In [12]:
# imports
from iot. iot_user import iot_comparison

## Step 2: Instantiate an `iot_comparison` class object

One can specify multiple parameterizations of tax policy to faciliate comparisons.  Below, we instantiate the `iot_comparison` class object by point to two differnt parameterizations of tax law: current law in 2017 (before the TCJA was enacted) and policies from President Biden's 2020 campaign.

In [13]:
iot1 = iot_comparison(
    policies=[
        "https://raw.githubusercontent.com/PSLmodels/examples/main/psl_examples/taxcalc/2017_law.json",
        "https://raw.githubusercontent.com/PSLmodels/examples/main/psl_examples/taxcalc/Biden2020.json",
    ],
    labels=["2017 Law", "Biden 2020"],
)

/Users/jason.debacker/repos/InverseOptimalTax/iot/inverse_optimal_tax.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



The the cell above is executed, Tax-Calculator is run for all of the specified policies, as well as its current law baseline.  With the results of those Tax-Calculator runs, the `iot` model then reverse enginers the social welfare functions that would be optimizes from each of those specifications of tax law.

## Accessing the results of the model

The object we created, `iot1`, has as attributes, all of the inputs to the inverse optimal tax calculation, as well as the outputs.  One can see all of these together in the `df` attribute of the `IOT` class that is an attribute of out `iot1` object created above. Note that because we have a list of policies, we can slice that list to reference results from a specific policy. The current law baseline will be the first object in this list, followed by the polcies we specified above in the order we specified them.

For instance, to reference a Pandas DataFrame with the input and outputs of the inverse optimal tax model of the 2020 Biden campaign proposals, we would do:

In [14]:
iot1.iot[-1].df()

,z,f,f_prime,mtr,mtr_prime,theta_z,g_z
0,68.465735,0.005217,1.597201e-07,0.107299,0.000003,1.002096,1.030166
1,1502.115716,0.005446,1.585990e-07,0.110921,0.000003,1.043745,1.033745
2,2509.463625,0.005606,1.573317e-07,0.113445,0.000002,1.070431,1.036229
3,3557.106423,0.005771,1.558448e-07,0.116051,0.000002,1.096066,1.038787
4,4492.316478,0.005916,1.541371e-07,0.118362,0.000002,1.117037,1.041038
...,...,...,...,...,...,...,...
495,495725.338326,0.000037,-3.179711e-10,0.448401,0.000001,-3.207945,0.863483
496,496620.081349,0.000037,-3.112711e-10,0.449534,0.000001,-3.158297,0.877669
497,497328.230151,0.000037,-3.038043e-10,0.450436,0.000001,-3.088577,0.896648
498,498447.640283,0.000037,-2.952215e-10,0.451876,0.000001,-3.018999,0.916768


## Visualizing results

It's often easiest to interpret their results through some visualizations.  the `iot_comparison` class makes this easy, with built in `plotly` plotting functions.

Let's plot the generalized social welfare function weights implied by the three policies (2021 current law, 2017 law, and the 2020 Biden campaign proposals):

In [17]:
iot1.plot()

The plot above is showing us the social welfare function weights, $g_z$, for taxpayers across the income distribution.  Higher $g_z$ imply a higher weight in the social welfare function that is optimized with that particular tax policy.  Roughly speaking, $g_z$ tells us how much the social welfare function would increase if we gave a taxpayer one more dollar of after tax income.

All three of the policies compared suggest social welfare functions that place more weight on tax payers with lower incomes.  The social welfare functions begin to diverge for tax payers with more than \\$200,000 in income.  Amongst the three policies, 2017 law provides the lowest weights on very high income taxpayers.  The TCJA, consistent with the regressive distributional estimates, reveals relatively higher weights for the highest income taxpayers.  Somewhat surprisingly, the Biden 2020 campaign proposal have the highest weights on those high income taxpayers of the policies compared.

## Plotting inputs to the calculation

We can also use the `iot_comparison` class' plotting function to help us understand why we see the implied social welfare weights we do.  

For instance, an important input in determining the social welfare weights is the marginal tax rate schedule.  Let's plot this for the three policies compared above:

In [18]:
iot1.plot(var="mtr")

We aer see are that each set of policies shows tax progressivity.  Marginal tax rates increase with income.  But beyond about \\$200,000 in income, these marginal tax rate functions look very different.  2017 law shows the slope of the marginal tax rates gradually leveling off.  In contrast, the current law baseline and Biden's 2020 campaign policies have marginal tax rates remaining relatively conant from about \\$200,000 to \\$350,000 in income and then an increasing slope after that.

We can plot the derivative of these marginal tax rate schedules to see this more clealy:

In [20]:
iot1.plot(var="mtr_prime")